Merge temperature, humidity data and annotation

In [ ]:
import csv
from datetime import datetime, timedelta

datetime_data = []
temp_data = []
humd_data = []
annot_data = []

with open("test_temp.csv", newline='') as f:
  reader = csv.reader(f, delimiter=',')
  for r in reader:
    datetime_data.append(datetime.fromisoformat(r[0]))
    temp_data.append(float(r[1]))

with open("test_humd.csv", newline='') as f:
  reader = csv.reader(f, delimiter=',')
  for r in reader:
    humd_data.append(float(r[1]))

with open("test_annot.csv", newline='') as f:
  reader = csv.reader(f, delimiter=',')
  for r in reader:
    annot_data.append(datetime.fromisoformat(r[0]))

annot_res = []
match_count = 0
for i, dt in enumerate(datetime_data):
  found_match = False
  for annot in annot_data:
    diff = annot - dt
    if diff == timedelta(0, 0, 0, 0, 0, 0, 0):
      annot_res.append(1)
      found_match = True
      match_count += 1
      break
  if not found_match:
    annot_res.append(0)

assert match_count == len(annot_data)

with open('test_data.csv', 'w', newline='') as f:
  writer = csv.writer(f)
  for i in range(len(datetime_data)):
    writer.writerow([datetime_data[i], temp_data[i], humd_data[i], annot_res[i]])

Preprocess data

In [ ]:
import numpy as np
from utils import preprocess

N = 18
data_batch = []
label = []

d_5ger, t_5ger, h_5ger, a_5ger, _ = preprocess("dataset/new/5ail_data.csv")
for i in range(len(d_5ger) - (N - 1)):
    data = []                               #  shape(4, N) ((feature, label), seq)
    drop = False
    label = a_5ger[i:i+N][N - 1]
    if label == 0:
        if np.random.random() >= 0.25:
            continue
    data.append(d_5ger[i:i+N])
    data.append(t_5ger[i:i+N])
    data.append(h_5ger[i:i+N])
    data.append([label for _ in range(N)])
    data_T = np.array(data).T               #  shape(N, 4) (seq, (feature, label))
    data_batch.append(data_T)
d_zurc, t_zurc, h_zurc, a_zurc, _ = preprocess("dataset/new/zuraach_data.csv")
for i in range(len(d_zurc) - (N - 1)):
    data = []                               #  shape(4, N) ((feature, label), seq)
    drop = False
    label = a_zurc[i:i+N][N - 1]
    if label == 0:
        if np.random.random() >= 0.25:
            continue
    data.append(d_zurc[i:i+N])
    data.append(t_zurc[i:i+N])
    data.append(h_zurc[i:i+N])
    data.append([label for _ in range(N)])
    data_T = np.array(data).T               #  shape(N, 4) (seq, (feature, label))
    data_batch.append(data_T)

data_batch = np.array(data_batch)           #  shape(N, seq, (feature, label))
label, count = np.unique(data_batch[:, :, -1][:, 0], return_counts=True)
print(label, count / count.sum(), count, len(data_batch))

np.save('dataset.npy', data_batch)